In [129]:
test = """#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#"""
source = test

In [130]:
source = open("./day_24.txt").read()

In [131]:
import numpy
cells = [(xtuple[0], ytuple[0], xtuple[1])  for ytuple in enumerate(source.splitlines()) for xtuple in enumerate(ytuple[1])]

height = len(source.splitlines())
width = len(source.splitlines()[0])
fieldWidth = width -2
fieldHeight = height -2
startPos = (1,0)
endPos = (fieldWidth, height-1)

dirdict = {
    ">": (1, 0),
    "<": (-1, 0),
    "^": (0, -1),
    "v": (0, 1),
}
storms = numpy.array([((cell[0], cell[1]), dirdict.get(cell[2])) for cell in cells if cell[2] in dirdict])
stormPositions = storms[:,0]
stormDirections = storms[:, 1]

In [132]:
def viz_storms(stormPositions, myposition):
    field = numpy.zeros((height, width), dtype=numpy.int8)
    for pos in stormPositions:
        field[pos[1], pos[0]] = 1
    for x in range(width):
        field[0,x] = 9
        field[height-1,x] = 9
    for y in range(height):
        field[y,0] = 9
        field[y,width-1] = 9
    field[startPos[1], startPos[0]] = 8
    field[0,0] = 9
    field[1,0] = 9
    field[0,2] = 9
    field[endPos[1], endPos[0]] = 7
    field[myposition[1], myposition[0]] = 2
    print("\n".join(["".join([" " if cell == 0 else str(cell) for cell in row]) for row in field]))

SPACE = " "
WALL  = "#"
def viz_storms_ascii(stormPositions, myPosition):
    tuplePositions = [tuple(pos) for pos in stormPositions]
    out = "Starting with e of " + str(myPosition) + "\n"
    for y in range(height):
        for x in range(width):
            if(x,y) == tuple(myPosition):
                out = out + "E"
            elif (x,y) == startPos:
                out = out + SPACE
            elif (x,y) == endPos:
                out = out + SPACE
            elif (x,y) in tuplePositions:
                possibles = [i for i in enumerate(tuplePositions) if i[1] == (x , y)]
                if len(possibles) == 1:
                    index = possibles[0][0]
                    stormDirection = tuple(stormDirections[index])
                    dirkey = list(dirdict.keys())[list(dirdict.values()).index(stormDirection)]
                    out = out + dirkey
                else:
                    out = out + str(len(possibles))
            elif x == 0 or y == 0 or x == width-1 or y == height-1:
                out = out + WALL
            else:
                out = out + SPACE
        out = out + "\n"
    print(out)

In [133]:
from time import sleep
from IPython.display import clear_output
moves = [(-1,0),(1,0), (0,1), (0,-1), (0,0)]
def travel(stormPositions, startPos, endPos):
    stormdict = {}
    possibles = [startPos]
    counter = 0
    win = False
    while win == False:
        if(len(possibles) == 0):
            print("Out of choices")
            break
        newStormPositions = numpy.mod(stormPositions +  (-1,-1) + stormDirections, (fieldWidth, height-2)) + (1,1)
        stormPositions = newStormPositions
        stormdict = {}
        for i, pos in enumerate(stormPositions):
            stormdict[tuple(pos)] = stormdict.get(tuple(pos), 0) + 1
        newpossibles = []
        for pos in possibles:
            if(tuple(pos) == endPos):
                win = True
                break
            for move in moves:
                newPos = tuple(numpy.add(pos,move))
                x,y = newPos
                if(x < 1 or y < 1 or x > fieldWidth or y > fieldHeight) and tuple(newPos) != endPos and tuple(newPos) != startPos: continue
                if(stormdict.get(tuple(newPos)) != None):continue
                if newPos not in newpossibles: newpossibles.append(newPos)
        if win == False: counter = counter + 1
        possibles = list(set(newpossibles))
    return counter, stormPositions


## Part 1: There

In [134]:

steps = travel(stormPositions.copy(), startPos, endPos)
print("Part 1 answer is", steps[0])

Part 1 answer is 240


## Part 2: ...and Back again, and there again

In [135]:

thereSteps, stormPositions = travel(stormPositions.copy(), startPos, endPos)
backAgainSteps, stormPositions = travel(stormPositions, endPos, startPos)
thereAgainSteps, stormPositions = travel(stormPositions, startPos, endPos)
print("Part 2 answer is", thereSteps + backAgainSteps + thereAgainSteps + 2)

Part 2 answer is 717
